In [1]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
#!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

!pip install -q bitsandbytes
!pip install -q evaluate jsonlines rouge_score bert-score
!pip install transformers peft accelerate bitsandbytes jsonlines
!pip install evaluate gradio
import evaluate


In [2]:
from transformers import AutoModel, AutoTokenizer

max_seq_length = 2048  # Choose any! We auto support ROPE scaling internally!
dtype = None  # None for auto detection. Float16 for Tesla T4, V100, bFloat16 for Ampere+

model_name_or_path = "jacopoda/lora_model"

from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name_or_path,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=True,
    # token = "hf_...", #se il nostro modello non è public
    # Use one if using gated models like meta-llama/Llama-2-7b-hf
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.12.4: Fast Llama patching. Transformers:4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [3]:
import gradio as gr

# Mock data for demonstration purposes
current_question = {"question": "", "answer": ""}

def generate_question(argument):
    """Generates a question and answer based on the provided argument."""
    # Mock question generation logic for simplicity
    question = f"What is an important aspect of {argument}?"
    answer = f"{argument} is important because it demonstrates the ability to apply knowledge to practical scenarios."
    current_question["question"] = question
    current_question["answer"] = answer
    return question

def evaluate_answer(user_answer):
    """Evaluates the user's answer without showing the correct answer."""
    feedback = "Your answer has been submitted! Click 'Reveal Answer' to see the correct answer."
    return feedback

def reveal_answer():
    """Reveals the correct answer."""
    return f"Correct Answer: {current_question['answer']}"

def reset_fields():
    """Resets the question, answer, and feedback fields for a new topic."""
    current_question["question"] = ""
    current_question["answer"] = ""
    return "", "", "", ""

# Set up the Gradio interface
with gr.Blocks() as interactive_quiz:
    gr.Markdown("## 🧠 Interactive Quiz")
    gr.Markdown("Provide a topic, answer a generated question, and reveal the correct answer!")

    with gr.Row():
        argument_input = gr.Textbox(label="Topic or Argument", placeholder="E.g., Machine Learning")
        generate_btn = gr.Button("Generate Question")
        change_topic_btn = gr.Button("Change Topic")

    question_display = gr.Textbox(label="Generated Question", interactive=False)
    user_answer_input = gr.Textbox(label="Your Answer", placeholder="Type your answer here...")
    evaluate_btn = gr.Button("Submit Answer")
    reveal_answer_btn = gr.Button("Reveal Answer")
    feedback_output = gr.HTML(label="Feedback")
    answer_output = gr.HTML(label="Correct Answer")

    # Define interactions
    generate_btn.click(
        generate_question,
        inputs=argument_input,
        outputs=question_display
    )

    evaluate_btn.click(
        evaluate_answer,
        inputs=user_answer_input,
        outputs=feedback_output
    )

    reveal_answer_btn.click(
        reveal_answer,
        inputs=[],
        outputs=answer_output
    )

    # Reset fields when changing the topic
    change_topic_btn.click(
        reset_fields,
        inputs=[],
        outputs=[question_display, user_answer_input, feedback_output, answer_output]
    )

# Launch the Gradio interface
interactive_quiz.launch(share=True, debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://c462ef76e3c433c9b6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://c462ef76e3c433c9b6.gradio.live
